In [1]:
from mlagents_envs.environment import UnityEnvironment  # Import Unity environment
from mlagents_envs.envs.unity_aec_env import UnityAECEnv
from mlagents_envs.envs.unity_parallel_env import UnityParallelEnv
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
from comunication_channel import AgentLogChannel
import neat
import os
import datetime
import pickle
import numpy as np

c:\Users\alesm\miniconda3\envs\mlagents\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
engine_config_channel = EngineConfigurationChannel()
engine_config_channel.set_configuration_parameters(time_scale=1)
agent_count_channel = AgentLogChannel()

env_path = "./Builds/train-env/autonomous-drone.exe"
save_nn_destination = 'result/best.pkl'

env = UnityEnvironment(file_name=None, worker_id=0, no_graphics=False, side_channels=[engine_config_channel, agent_count_channel])
env = UnityAECEnv(env)
env.reset()
num_agents = len(env.possible_agents)

num_actions = env.action_space(env.possible_agents[0]).shape[0]
print(f"There is total of {num_actions} actions in enviroment")
num_inputs = env.observation_space(env.possible_agents[0]).shape[0]
print(f"There is total of {num_inputs} inputs in enviroment")

MAX_STEPS = 1200
NUM_TRIES = 3

There is total of 4 actions in enviroment
There is total of 14 inputs in enviroment


In [3]:

def map_agents():
    map = {}
    current_index = 0
    for agent in env.agents:
        map[int(agent.split("=")[2])] = current_index
        current_index += 1
    return map

def set_agents_and_double_reset(num_agents: int):
    agent_count_channel.send_int(data=num_agents) 
    env.reset()
    env.reset()

def create_policies(genomes, cfg):
    policies = []
    for _, g in genomes:
        policy = neat.nn.FeedForwardNetwork.create(g, cfg)
        policies.append(policy)
        g.fitness = 0
    return policies

def eval_genome(genome, cfg):
    policies = [neat.nn.FeedForwardNetwork.create(genome, cfg)] * env.num_agents
    map = map_agents()
    for _ in range(NUM_TRIES):
        rewards = [0] * env.num_agents
        for agent in env.agent_iter(env.num_agents * MAX_STEPS):
            current_agent = int(agent.split("=")[2])
            obs, reward, done, info = env.last(observe=True)
            if done:
                action = None
            else:
                action = np.asarray(policies[map[current_agent]].activate(obs))
            rewards[map[current_agent]] += reward
            env.step(action)
        env.reset()
        print(rewards)

In [4]:

config_path = 'test_config'
with open(save_nn_destination, "rb") as f:
    genome = pickle.load(f)
    print(genome)
print(genome.fitness)
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                    neat.DefaultSpeciesSet, neat.DefaultStagnation,
                    config_path)

eval_genome(genome, config)
env.close()

Key: 232848
Fitness: 370.02201570943
Nodes:
	0 DefaultNodeGene(key=0, bias=0.08342484128306193, response=0.8554439664794514, activation=tanh3, aggregation=sum)
	1 DefaultNodeGene(key=1, bias=-0.4363384675321179, response=1.033066348948434, activation=tanh3, aggregation=sum)
	2 DefaultNodeGene(key=2, bias=0.7366637574982908, response=-6.760282982433005, activation=tanh3, aggregation=sum)
	3 DefaultNodeGene(key=3, bias=-0.4950450902420258, response=-3.351830415439236, activation=tanh3, aggregation=sum)
	12497 DefaultNodeGene(key=12497, bias=1.1616049560753554, response=-4.4976058974527575, activation=tanh3, aggregation=sum)
	14671 DefaultNodeGene(key=14671, bias=-0.1285704285253702, response=4.19866930383561, activation=tanh3, aggregation=sum)
	16195 DefaultNodeGene(key=16195, bias=-0.3041954580903855, response=-2.8784388519662616, activation=tanh3, aggregation=sum)
	16211 DefaultNodeGene(key=16211, bias=0.4298147627663516, response=-4.015348514948702, activation=tanh3, aggregation=sum)
